# Download the buildings hexagrid data

Question: How many buildings were mapped with RapID (Microsoft buildings) in every continent?

#### Import libs

In [1]:
import json
import os
import requests

### Get the data for every specified region

#### Definitions

In [2]:
# define the dictinaries

data_dic = {0: "blds_india_18-23_ai.json",
            1: "blds_vietnam_18-23_ai.json",
            2: "blds_tanzania_18-23_ai.json",
            3: "blds_kenya_18-23_ai.json",
            4: "blds_nigeria_18-23_ai.json", }

geojson_folder_dic = {0: "geojson-continent-states",
                      1: "geojson-africa-states",
                      2: "geojson-northamerica-states",
                      3: "geojson-europe-states",
                      4: "geojson-asia-states",}

In [3]:
# define the user input

while True:
    try:
        user_d_spec = int(input("Which data do you want to download?\
                            Enter 0 for continents (2018–2023),\
                            1 for Africa states (2018–2023),\
                            2 for North-Central America states (2018–2023)\
                            3 for Europe states (2018–2023)\
                            4 for Asia states (2018–2023)"
                                ))
    except ValueError:
        print("Please enter a number.")
        continue
    if user_d_spec not in range(len(geojson_folder_dic)):
        print("Please enter 0, 1, 2, 3 or 4.")
        continue
    break


Please enter 0, 1, 2, 3 or 4.


In [38]:
def convert_geojson_structure(original_dict):
    new_dict = {}
    new_dict["type"] = "FeatureCollection"
    new_dict["features"] = []

    feature_dict = {}
    feature_dict["type"] = "Feature"
    feature_dict["geometry"] = {}
    feature_dict["geometry"]["type"] = "MultiPolygon"


    new_dict["features"].append(feature_dict)
    
    feature_dict["geometry"]["coordinates"] = original_dict["geometry"]["coordinates"]
    # feature_dict["geometry"]["type"] = "MultiPolygon"

    return new_dict


In [2]:
# test

def convert_geojson_structure(original_dict):
    new_dict = {}
    new_dict["type"] = "FeatureCollection"
    new_dict["features"] = []

    feature_dict = {}
    feature_dict["type"] = "Feature"
    feature_dict["geometry"] = {}
    feature_dict["geometry"]["type"] = "MultiPolygon"

    new_dict["features"].append(feature_dict)

    feature_dict["geometry"]["coordinates"] = original_dict["geometry"]["coordinates"]
    # feature_dict["geometry"]["type"] = "MultiPolygon"
    feature_dict["properties"] = original_dict["properties"]


    return new_dict


In [3]:

os.chdir('..')
home_dir = os.getcwd()
geojson_regions = os.path.join(home_dir, "geojson-regions", "")

with open(f"{geojson_regions}geojson-africa-states.geojson", "r") as file:
    bpolys = json.load(file)


In [4]:
for f in bpolys["features"][0:3]:
    print(convert_geojson_structure(f)["features"][0]["properties"]["NAME_EN"])
    print(convert_geojson_structure(f))


Ethiopia
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[34.070697863000134, 9.454592111000068], [34.06689172400007, 9.531176453000072], [34.09820764200009, 9.679720358000026], [34.22192102100007, 10.026029968000032], [34.28083215300012, 10.080109355000019], [34.30346643000013, 10.113544006000097], [34.30501672400004, 10.124602763000013], [34.30160607900012, 10.14899403900003], [34.301812785000095, 10.160285340000101], [34.30460331200004, 10.165633850000049], [34.31421512800006, 10.175452372000024], [34.31741906800005, 10.181265971000101], [34.32589400200004, 10.223563131000091], [34.32486047400005, 10.26865081800014], [34.276181275000056, 10.488714905000037], [34.27173710100004, 10.530081889000073], [34.279488566000055, 10.565506083000102], [34.36558150200011, 10.685782980000084], [34.40423547400013, 10.759835307000102], [34.420048462000125, 10.780815938000103], [34.43730839000011, 10.795672913000132], [34.55781783

In [5]:
bpolys["features"]


[{'type': 'Feature',
  'properties': {'SOVEREIGNT': 'Ethiopia',
   'LEVEL': 2,
   'TYPE': 'Sovereign country',
   'ADMIN': 'Ethiopia',
   'NAME': 'Ethiopia',
   'CONTINENT': 'Africa',
   'NAME_EN': 'Ethiopia'},
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[[[34.070697863000134, 9.454592111000068],
      [34.06689172400007, 9.531176453000072],
      [34.09820764200009, 9.679720358000026],
      [34.22192102100007, 10.026029968000032],
      [34.28083215300012, 10.080109355000019],
      [34.30346643000013, 10.113544006000097],
      [34.30501672400004, 10.124602763000013],
      [34.30160607900012, 10.14899403900003],
      [34.301812785000095, 10.160285340000101],
      [34.30460331200004, 10.165633850000049],
      [34.31421512800006, 10.175452372000024],
      [34.31741906800005, 10.181265971000101],
      [34.32589400200004, 10.223563131000091],
      [34.32486047400005, 10.26865081800014],
      [34.276181275000056, 10.488714905000037],
      [34.27173710100004, 10.530

In [14]:
data = []

for i, feature in enumerate(bpolys["features"][0:2]):

    desired_geojson_structure = convert_geojson_structure(feature)

    parameters = {
        "bpolys": json.dumps(desired_geojson_structure),  # pass GeoJSON as string.
        "filter": "building=* and building!=no and geometry:polygon",
        "groupByKey": "source",
        "groupByValues": "microsoft/BuildingFootprints",
        "format": "json",
        "time": "2018-01-01/2023-01-01/P1Y",
    }
    headers = {
        "accept": "application/json",
        "Content-Type": "application/x-www-form-urlencoded",
    }

    for value in parameters.values():
        assert value != "", "Please provide values for the parameters"

    response = requests.post(url, data=parameters, headers=headers)
    response.raise_for_status()  # Raise an Exception if HTTP Status Code is not 200

    print("Response:")
    print(desired_geojson_structure["features"][0]["properties"]["NAME_EN"])
    print(json.dumps(response.json(), indent=4))  # Pretty print response

    result = response.json()["groupByResult"]
    
    for state in result:
        state["groupByObject"][0] = desired_geojson_structure["features"][0]["properties"]["NAME_EN"]
    
    
    data.append(result)
    

Response:
Ethiopia
{
    "attribution": {
        "url": "https://ohsome.org/copyrights",
        "text": "\u00a9 OpenStreetMap contributors"
    },
    "apiVersion": "1.9.0",
    "groupByResult": [
        {
            "groupByObject": [
                "feature1",
                "remainder"
            ],
            "result": [
                {
                    "timestamp": "2018-01-01T00:00:00Z",
                    "value": 284919.0
                },
                {
                    "timestamp": "2019-01-01T00:00:00Z",
                    "value": 350579.0
                },
                {
                    "timestamp": "2020-01-01T00:00:00Z",
                    "value": 395155.0
                },
                {
                    "timestamp": "2021-01-01T00:00:00Z",
                    "value": 470680.0
                },
                {
                    "timestamp": "2022-01-01T00:00:00Z",
                    "value": 629701.0
                },
     

In [19]:
one_feat_test = convert_geojson_structure(bpolys["features"][0])


In [21]:
for group in result:
    group["groupByObject"][0] = one_feat_test["features"][0]["properties"]["NAME_EN"]


In [22]:
result

[{'groupByObject': ['Ethiopia', 'remainder'],
  'result': [{'timestamp': '2018-01-01T00:00:00Z', 'value': 1007356.0},
   {'timestamp': '2019-01-01T00:00:00Z', 'value': 1025617.0},
   {'timestamp': '2020-01-01T00:00:00Z', 'value': 1042953.0},
   {'timestamp': '2021-01-01T00:00:00Z', 'value': 1221037.0},
   {'timestamp': '2022-01-01T00:00:00Z', 'value': 1653908.0},
   {'timestamp': '2023-01-01T00:00:00Z', 'value': 1788605.0}]},
 {'groupByObject': ['Ethiopia', 'source=microsoft/BuildingFootprints'],
  'result': [{'timestamp': '2018-01-01T00:00:00Z', 'value': 0.0},
   {'timestamp': '2019-01-01T00:00:00Z', 'value': 0.0},
   {'timestamp': '2020-01-01T00:00:00Z', 'value': 0.0},
   {'timestamp': '2021-01-01T00:00:00Z', 'value': 0.0},
   {'timestamp': '2022-01-01T00:00:00Z', 'value': 0.0},
   {'timestamp': '2023-01-01T00:00:00Z', 'value': 0.0}]}]

In [15]:
result[0]['groupByObject'][0]


'feature1'

In [12]:
for feature in bpolys["features"][0:2]:
    #print(feature["features"][0]["properties"]["NAME_EN"])
    print(convert_geojson_structure(feature))
    

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[34.070697863000134, 9.454592111000068], [34.06689172400007, 9.531176453000072], [34.09820764200009, 9.679720358000026], [34.22192102100007, 10.026029968000032], [34.28083215300012, 10.080109355000019], [34.30346643000013, 10.113544006000097], [34.30501672400004, 10.124602763000013], [34.30160607900012, 10.14899403900003], [34.301812785000095, 10.160285340000101], [34.30460331200004, 10.165633850000049], [34.31421512800006, 10.175452372000024], [34.31741906800005, 10.181265971000101], [34.32589400200004, 10.223563131000091], [34.32486047400005, 10.26865081800014], [34.276181275000056, 10.488714905000037], [34.27173710100004, 10.530081889000073], [34.279488566000055, 10.565506083000102], [34.36558150200011, 10.685782980000084], [34.40423547400013, 10.759835307000102], [34.420048462000125, 10.780815938000103], [34.43730839000011, 10.795672913000132], [34.55781783000015, 1

In [17]:
for item in result:
    for it in item["groupByObject"]:
        print(it)

feature1
remainder
feature1
source=microsoft/BuildingFootprints


In [4]:
# define the functions


def data_gen(geojson_folder_dic, name):

    with open(f"{geojson_folder_dic}\\{name}", "r") as file:
        bpolys = json.load(file)

    

    parameters = {
        "bpolys": json.dumps(bpolys),  # pass GeoJSON as string.
        "filter": "building=* and building!=no and geometry:polygon",
        "groupByKey": "source",
        "groupByValues": "microsoft/BuildingFootprints",
        "format": "json",
        "time": "2018-01-01/2023-01-01/P1Y",
    }
    headers = {
        "accept": "application/json",
        "Content-Type": "application/x-www-form-urlencoded",
    }

    for value in parameters.values():
        assert value != "", "Please provide values for the parameters"

    response = requests.post(url, data=parameters, headers=headers)
    response.raise_for_status()  # Raise an Exception if HTTP Status Code is not 200

    print("Response:")
    print(json.dumps(response.json(), indent=4))  # Pretty print response

    result = response.json()["groupByResult"]
    return result


def get_geojson_names(geojson_folder_dic):

    os.chdir('..')
    home_dir = os.getcwd()
    geojson_dir = os.path.join(home_dir, geojson_folder_dic, "")

    geojson_names = []
    for (dir_path, dir_names, file_names) in os.walk(geojson_dir):  # give the path
        geojson_names.extend(file_names)
    print(geojson_names)
    return geojson_names


In [7]:
# define the URL

base_url = "https://api.ohsome.org/v1"
endpoint = "/elements/count/groupBy/boundary/groupBy/tag"
url = base_url + endpoint

#### Get the data

In [6]:
data = []

for geojson_name in get_geojson_names(geojson_folder_dic[user_d_spec]):

    data.append(data_gen(geojson_folder_dic[user_d_spec], geojson_name))


['46_Egypt.geojson']
Response:
{
    "attribution": {
        "url": "https://ohsome.org/copyrights",
        "text": "\u00a9 OpenStreetMap contributors"
    },
    "apiVersion": "1.9.0",
    "groupByResult": [
        {
            "groupByObject": [
                "feature1",
                "remainder"
            ],
            "result": [
                {
                    "timestamp": "2018-01-01T00:00:00Z",
                    "value": 59622.0
                },
                {
                    "timestamp": "2019-01-01T00:00:00Z",
                    "value": 73296.0
                },
                {
                    "timestamp": "2020-01-01T00:00:00Z",
                    "value": 82020.0
                },
                {
                    "timestamp": "2021-01-01T00:00:00Z",
                    "value": 98403.0
                },
                {
                    "timestamp": "2022-01-01T00:00:00Z",
                    "value": 109751.0
                

In [7]:
# export the data

def export_data(name):
    with open(f"downloaded-data\\{name}", "w") as file:
        json.dump(data, file, indent=4)


export_data(data_dic[user_d_spec])
